In [17]:
# 1.1 data analysis and wrangling
import numpy as np
import pandas as pd
import matplotlib.pyplot as pltz

In [18]:
# load data
df_ratings=pd.read_csv('ratings.csv')
df_movie = pd.read_csv('movies.csv')
# data merge
df_movies = pd.merge(df_ratings, df_movie, on=['movieId'], how='left')

In [21]:
user_movie_ratings = pd.pivot_table(df_movies, index='userId', columns= ['movieId', 'title'], values='rating')

In [28]:
MD = user_movie_ratings.replace(np.nan, 0)
MD.sort_index(axis=1)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,My Friend Rockefeller (2015),Sunspring (2016),Kingsglaive: Final Fantasy XV (2016),Body (2015),Sharknado 4: The 4th Awakens (2016),The Last Brickmaker in America (2001),Stranger Things,Rustom (2016),Mohenjo Daro (2016),The Beatles: Eight Days a Week - The Touring Years (2016)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
class MatrixFact():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
        
    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))

    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost) / len(xi)

    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq

    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq

    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)

    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)
   
    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])

if __name__ == "__main__":
    R = MD.values
    factorizer = MatrixFact(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=300, verbose=True)
    factorizer.fit()
    factorizer.print_results()

Iteration: 10 ; cost = 0.0027
Iteration: 20 ; cost = 0.0026
Iteration: 30 ; cost = 0.0026
Iteration: 40 ; cost = 0.0025
Iteration: 50 ; cost = 0.0025
Iteration: 60 ; cost = 0.0025
Iteration: 70 ; cost = 0.0025
Iteration: 80 ; cost = 0.0024
Iteration: 90 ; cost = 0.0024
Iteration: 100 ; cost = 0.0024
Iteration: 110 ; cost = 0.0024
Iteration: 120 ; cost = 0.0024
Iteration: 130 ; cost = 0.0023
Iteration: 140 ; cost = 0.0023
Iteration: 150 ; cost = 0.0023
Iteration: 160 ; cost = 0.0023
Iteration: 170 ; cost = 0.0023
Iteration: 180 ; cost = 0.0023
Iteration: 190 ; cost = 0.0023
Iteration: 200 ; cost = 0.0023
Iteration: 210 ; cost = 0.0023
Iteration: 220 ; cost = 0.0023
Iteration: 230 ; cost = 0.0023
Iteration: 240 ; cost = 0.0023
Iteration: 250 ; cost = 0.0022
Iteration: 260 ; cost = 0.0022
Iteration: 270 ; cost = 0.0022
Iteration: 280 ; cost = 0.0022
Iteration: 290 ; cost = 0.0022
Iteration: 300 ; cost = 0.0022
User Latent P:
[[ 0.1503209  -0.44021358 -0.79794637]
 [ 0.26450305 -0.14379767

In [37]:
factorizer.get_complete_matrix()

array([[2.92899245, 2.82633557, 2.16758363, ..., 2.32139941, 2.14810908,
        4.01486475],
       [3.7851122 , 3.38220769, 3.0889252 , ..., 4.30152928, 2.75514738,
        5.11774981],
       [3.73827319, 3.22549049, 2.29794775, ..., 2.60325439, 2.57176931,
        4.40040058],
       ...,
       [3.91948165, 3.26404939, 2.80626313, ..., 4.03666388, 2.5851502 ,
        4.71272822],
       [3.4508508 , 2.80807026, 5.21111935, ..., 9.9799865 , 1.19806561,
        2.49831496],
       [3.91797113, 3.63178327, 3.01695904, ..., 3.79620473, 3.21798201,
        5.99383543]])